In [43]:
import pandas as pd
import numpy as np
import pylab 
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import os, sys

In [44]:
def subcategory_customer_week_month_churn_rate(filename):
    
    
    # You need to change yourself
    subcategory_name = os.path.splitext(filename)[0]   

    # In[5]:
    df_sales = pd.read_csv("../data/subcategory/"+ subcategory_name + ".csv")

    
    df_sales_date = df_sales
    df_sales_date['date']=pd.to_datetime(df_sales_date['order_date']).dt.strftime("%Y-%m-%d")
    df_sales_date=df_sales_date.sort_values(by=['date','address_hash']) 
    df_sales_171819 = df_sales_date
    df_sales_171819 = df_sales_171819.loc[(df_sales_171819['date'] >= '2017-01-01') & (df_sales_171819['date']!='2018-12-12')]


    # ### Getting individual's number of orders and end purchse date

    # In[24]:


    customers_orders = {}
    customers_start = {}
    customers_end = {}
    for index, row in df_sales_171819.iterrows():   
        if row['address_hash'] in customers_orders.keys():
            if row['date'] != customers_end[row['address_hash']] and         row['date'] != customers_start[row['address_hash']]:
                customers_orders[row['address_hash']] += 1 # update the amount of orders
                customers_end[row['address_hash']] = row['date'] # update the last order date
        else:
            customers_orders[row['address_hash']] = 1 
            customers_start[row['address_hash']] = row['date'] # record the start order date
            customers_end[row['address_hash']] = row['date'] # update the last order date


    # In[25]:


    # Transform dictionaries into dataframes and merge them according to keys
    df_customers_start = pd.DataFrame(customers_start.items(), columns=['customer', 'date_start'])
    df_customers_end = pd.DataFrame(customers_end.items(), columns=['customer', 'date_end'])


    # In[26]:


    df = pd.merge(df_customers_start, df_customers_end,                                            on='customer', how ='right')


    # In[27]:


    df_start_end_sort = df.sort_values(by=['date_start','date_end'])


    # ## Churn Rates

    # **<center>Churn Rate = Customers Left / (New Customers + Existing Customers)</center>** <br/>
    # Left customers judgement: His end purchase date plus 3 months is earlier than the last day of the month. 3 months is the largest subsciption interval allowed. <br/>
    # New customers judgement: His start purchase date is later than the first day of the month. <br><br/>
    # First let's decide the day that marks the leave of each customer and append it to the df_sort we got earlier. In this part, we still stick to the whole customer base instead of the "subscribers" market. 

    # #### daily churn

    # Let's first create a new dataframe with each day from 2017-1-1 to 2020-2-10 as the key column 

    # In[28]:


    # Getting number of days need to fill the column
    from datetime import date
    d0 = date(2017, 1, 1)
    d1 = date(2020, 2, 10)
    delta = d1 - d0


    # In[29]:


    # create an array of dates starting at '2017-01-01', one per day. and set day as index
    rng = pd.date_range('2017-01-01', periods=1135, freq='D')
    df_calender = pd.DataFrame({ 'Date': rng}) 


    # In[35]:




    i = 0
    for index, row in df_calender.iterrows():
        count = 0
        while i < df_start_end_sort.shape[0] and pd.to_datetime(df_start_end_sort.at[i,'date_start']) == pd.to_datetime(row['Date']):
            count += 1
            i += 1
            if i >= df_start_end_sort.shape[0]:
                break
        df_calender.at[index,'new_customers'] = count




    # In[36]:


    df_sort_by_date_end = df_start_end_sort.sort_values(by=['date_end'])


    # In[37]:


    i = 1
    for index, row in df_calender.iterrows():
        count = 0
        while pd.to_datetime(df_start_end_sort.at[i,'date_end']) == pd.to_datetime(row['Date']):
            count += 1
            i += 1
        df_calender.at[index,'new_churn'] = count


    # #### Now calculate the daily existing customer column
    # The first one or two months existing customer base may not be very accurate due to ignorance of active customers coming from the end of 2016. This error will gradually disappear after two months or so. Let's first regard the number of unique customers during December of 2016 as existing customers.

    # In[38]:


    df_sales_16_Dec = df_sales.loc[(df_sales['date'] >= '2016-12-01')                                 & (df_sales['date'] < '2017-01-01') ]


    # In[40]:


    existing_customer = df_sales_16_Dec['address_hash'].nunique()


    # In[41]:


    df_existing = df_calender


    # In[42]:


    df_existing.at[0,'existing_customers'] = existing_customer


    # In[43]:


    df_existing.at[0,'total_customers'] = existing_customer


    # In[44]:


    for i in range(1,df_existing.shape[0]):
        df_existing.at[i,'existing_customers'] = df_existing.at[i-1,'total_customers'] 
        df_existing.at[i,'total_customers'] = df_existing.at[i,'existing_customers']     + df_existing.at[i,'new_customers'] - df_existing.at[i,'new_churn']


    # #### Monthly/Weekly churn

    # <img src="churn_rate.png" width=500 height=500 />
    # 
    # https://www.profitwell.com/blog/the-complete-saas-guide-to-calculating-churn-rate-and-keeping-it-simple

    # In[45]:


    df_month_week_existing = df_existing


    # In[46]:


    df_month_week_existing['Month'] = pd.to_datetime(df_month_week_existing['Date']).dt.to_period('M')


    # In[47]:


    df_month_week_existing['Date'] = pd.to_datetime(df_month_week_existing['Date'], errors='coerce')
    df_month_week_existing['Week'] = df_month_week_existing['Date'].dt.strftime('%Y-%U')
    #df.formatted_date.apply(lambda x: x.weekofyear)


    # ### Calculate monthly/weekly churn rate is a new and more precise way
    # <img src="shopify_way.png" width=500 height=500 />
    # https://www.profitwell.com/blog/the-complete-saas-guide-to-calculating-churn-rate-and-keeping-it-simple

    # In[48]:


    df_month_exist = df_month_week_existing.groupby(['Month']).agg({'new_churn':'sum','existing_customers':'mean'})


    # In[49]:


    df_month_exist['monthly_churn_rate'] = df_month_exist['new_churn']/df_month_exist['existing_customers'] 


    # In[50]:


    df_week_exist = df_month_week_existing.groupby(['Week']).agg({'new_churn':'sum','existing_customers':'mean'})


    # In[51]:


    df_week_exist['weekly_churn_rate'] = df_week_exist['new_churn']/df_week_exist['existing_customers'] 


# In[52]:

    dfs = {subcategory_name + "_monthly_churn_rate":df_month_exist, subcategory_name + "_weekly_churn_rate":df_week_exist} 

    for k, v in dfs.items():
        path_name = '../output/subcategory_churn_rate/customers/' + subcategory_name + '_cus_churn/'
        if not os.path.exists(path_name):
            os.makedirs(path_name)
        v.to_csv(path_name + '{}.csv'.format(k))

In [45]:
for filename in os.listdir("../data/subcategory/"):
    subcategory_customer_week_month_churn_rate(filename)
#     print(os.path.splitext(filename))
#     subcategory_subscription_week_month_churn_rate(filename)

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


KeyError: 'order_date'